## Connect to Weaviate

In [ ]:
import weaviate

client = weaviate.Client(
  url="https://llamaindex-test-hopqzn3c.weaviate.network",  # URL to Weaviate instance
)

client.schema.get()  # Get the schema to test connection

## Create Schema

In [ ]:
schema = {
   "classes": [
       {
           "class": "BlogPost",
           "description": "Blog post from the Weaviate website.",
           "vectorizer": "text2vec-openai",
           "moduleConfig": {
               "generative-openai": { 
                    "model": "gpt-3.5-turbo"
                }
           },
           "properties": [
               {
                  "name": "Content",
                  "dataType": ["text"],
                  "description": "Content from the blog post",
               }
            ]
        }
    ]
}

client.schema.delete_all()

client.schema.create(schema)

print("Schema was created.")

## Load in Data

In [ ]:
from llama_index import download_loader, SimpleWebPageReader

SimpleWebPageReader = download_loader("SimpleWebPageReader")

loader = SimpleWebPageReader()
blog = loader.load_data(urls=['https://weaviate.io/blog/llamaindex-and-weaviate'])

## Parse the Documents 

In [ ]:
from llama_index.node_parser import SimpleNodeParser

parser = SimpleNodeParser()

nodes = parser.get_nodes_from_documents(blog)

## Construct Vector Store

In [ ]:
from llama_index.vector_stores import WeaviateVectorStore
from llama_index import VectorStoreIndex, StorageContext
from llama_index.storage.storage_context import StorageContext
import os

os.environ["OPENAI_API_KEY"] = "sk-key" 

# construct vector store
vector_store = WeaviateVectorStore(weaviate_client = client, index_name="BlogPost", text_key="content")

# setting up the storage for the embeddings
storage_context = StorageContext.from_defaults(vector_store = vector_store)

# set up the index
index = VectorStoreIndex(nodes, storage_context = storage_context)

query_engine = index.as_query_engine()

## Set up Sub Question Query Engine

In [ ]:
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine 

query_engine_tools = [
    QueryEngineTool(
        query_engine = query_engine, 
        metadata = ToolMetadata(name='BlogPost', description='Blog post about the integration of LlamaIndex and Weaviate')
    )
]

query_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

## Query Time

In [ ]:
response = await query_engine.aquery('How does LlamaIndex help data indexing in Weaviate?')

In [ ]:
print(response)